In [1]:
import pandas as pd
import numpy as np
import scipy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from statsmodels.stats.weightstats import *

# Параметрические критерии
## Разрушители легенд зевают
В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.
### Ответ
Выборки независимые, так как разные люди в них участвовали. Зевота долна быть заразительна, поэтому среднее экспериментальной выборки должно быть больше среднего контрольной выборки.

  исход| $X_1$ | $X_2$  
  ------------- | -------------|-------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$

$$H_0: \mu_1 = \mu_2$$
$$H_1:\mu_1 >  \mu_2$$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$
$$p_{value}(T) \sim  N(0,1)$$

In [2]:
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [3]:
success_1 = 10 
n_1 = 34
people_1 = [1] * success_1 + [0] * (n_1 - success_1)

success_2 = 4 
n_2 = 16
people_2 = [1] * success_2 + [0] * (n_2 - success_2)

In [4]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_ind(people_1, people_2))

z_stat = proportions_diff_z_stat_ind(people_1, people_2)
p_value = proportions_diff_z_test(z_stat, 'greater')
print("z_stat = ", z_stat)
print("p_value = ", np.round(p_value, 4))

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]
z_stat =  0.32410186177608225
p_value =  0.3729


In [5]:
print('Ответ от scipy',scipy.stats.ttest_ind(people_1, people_2)[1]/2, 
      'отличается от ручного ответа.', np.round(p_value, 4),
      'Скорее всего отличаются статистики у этих методов.')

Ответ от scipy 0.3759744743336366 отличается от ручного ответа. 0.3729 Скорее всего отличаются статистики у этих методов.


Достигаемый уровень значимости больше порога в 0.05 - значит нулевая гипотеза не отклоняется. Статистическая значимость гласит, что среднее выборок одинаково, даже ноль входит в доверительный интервал. Практическая значимость от разрушителей мифов показывает обратное.

## Подделываем банкноты
Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными. 

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.model_selection.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

1. логистическая регрессия по признакам $X_1,X_2,X_3$ 
2. логистическая регрессия по признакам $X_4,X_5,X_6$

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили $5.5\times10^{-8}$, нужно ввести 8).

### Ответ
Выборки связные, так как в выборках одни и те же объекты. Альтернатива двусторонняя. Среднее ошибок первого классификатора равно среднему ошибок второго классификатора?

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------| -------------| -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  

$$H_0: \mu_1 = \mu_2$$
$$H_1:\mu_1 <\neq>  \mu_2$$


$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$
$$p_{value}(T) \sim  N(0,1)$$

In [6]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = list(zip(sample1, sample2))
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [7]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = list(zip(sample1, sample2))
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [8]:
df = pd.read_csv('banknotes.txt', header = 0, sep = '\t')
df.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 7 columns):
X1      200 non-null float64
X2      200 non-null float64
X3      200 non-null float64
X4      200 non-null float64
X5      200 non-null float64
X6      200 non-null float64
real    200 non-null int64
dtypes: float64(6), int64(1)
memory usage: 11.0 KB


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df, df.real, test_size=50,train_size=150,random_state=1)
clf1 = LogisticRegression(random_state=0).fit(X_train[['X1','X2','X3']], y_train)
clf2 = LogisticRegression(random_state=0).fit(X_train[['X4','X5','X6']], y_train)
error_1 = np.abs(clf1.predict(X_test[['X1','X2','X3']]) - y_test)
error_2 = np.abs(clf2.predict(X_test[['X4','X5','X6']]) - y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
print("95%% confidence interval for a difference between proportions: [%f, %f]" %\
      proportions_diff_confint_rel(error_1, error_2))

z_stat = proportions_diff_z_stat_rel(error_1, error_2)
p_value = proportions_diff_z_test(z_stat)
print("z_stat = ", z_stat)
print("p_value = ", np.round(p_value, 4))

95% confidence interval for a difference between proportions: [0.059945, 0.300055]
z_stat =  2.9386041680175268
p_value =  0.0033


In [12]:
print('Ответ с помощью scipy ', np.round(scipy.stats.ttest_ind(error_1, error_2)[1], 4),
     'отличается от ручного ответа', np.round(p_value, 4),
      '. Скорее всего отличаются статистики у этих методов.')

Ответ с помощью scipy  0.0037 отличается от ручного ответа 0.0033 . Скорее всего отличаются статистики у этих методов.


Достигаемый уровень значимости меньше порога в 0.05 - значит нулевая гипотеза отклоняется.

## Эффективность программы обучения
Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.
### Ответ
Смотрим одну выборку. Знаем мат ожидание и отклонение генеральной совокупности. Альтернатива одностороняя.
$$H_0: \mu = \mu_0=525$$
$$H_1:\mu > \mu_0$$
$$T: Z({X}) = \frac{(\hat{X}-\mu_0)}{\frac{\sigma}{\sqrt{n}}}$$
$$p_{value}(T) \sim  N(0,1)$$

In [13]:
n = 200000
mu = 525
sigma = 100

n_course = 100
mu_course = 541.4

In [14]:
Z = (mu_course - mu)/(sigma / np.sqrt(n_course))
np.round(proportions_diff_z_test(Z, 'greater'), 4)

0.0505

In [15]:
mu_course = 541.5
Z = (mu_course - mu)/(sigma / np.sqrt(n_course))
np.round(proportions_diff_z_test(Z, 'greater'), 4)

0.0495

При $\mu_{course} = 541.4 $ нулевая гипотеза не отвергается. При $\mu_{course} = 541.5 $ нулевая гипотеза отвергается. Небольшое отличие - а из-за него при уровне значимости 0.05 выбранные гипотезы отличаются.

## Результат
Был закреплен материал из лекций. 